# Simple CNN on MNIST - Security Due to Overfitting

In [1]:
import numpy as np

import os

import keras
from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model

import tensorflow as tf

from keras.callbacks import TensorBoard

from cleverhans.utils_mnist import data_mnist
from cleverhans.attacks import (BasicIterativeMethod, CarliniWagnerL2, DeepFool, ElasticNetMethod, 
                                FastFeatureAdversaries, FastGradientMethod, LBFGS, MadryEtAl, 
                                MomentumIterativeMethod, SPSA, SaliencyMapMethod, VirtualAdversarialMethod)
from cleverhans.utils_keras import KerasModelWrapper

from ipywidgets import interact
import ipywidgets as widgets

/home/Chris/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [3]:
from_saved_model = True
run_attack = True

## Configurable Parameters

In [4]:
# Used in Getting the Data
train_start=0
train_end=60000
test_start=0
test_end=10000

attack_start=0
attack_end=100

batch_size = 128
num_classes = 10
epochs = 100
input_shape = (28, 28, 1)

num_points = 10
attack_names = ['basic_iterative', 'fast_gradient', 'madry', 'momentum_iterative']

run_ident = '7'
callbacks = [TensorBoard(log_dir='./logs/overfitting_r{}'.format(run_ident))]

## Setup Tensorflow Session

In [5]:
keras.layers.core.K.set_learning_phase(0)

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)

# Create TF session and set as Keras backend session
sess = tf.Session()
keras.backend.set_session(sess)

## Get the MNIST Data

In [6]:
X_train, Y_train, X_test, Y_test = data_mnist(train_start=train_start,
                                              train_end=train_end,
                                              test_start=test_start,
                                              test_end=test_end)

Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)


## Create the CNN Architecture

In [7]:
# Define input TF placeholder
x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y = tf.placeholder(tf.float32, shape=(None, 10))

# Define TF model graph

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
preds = model(x)

## Setup Adversarial Generation Class

In [8]:
class GenAdv(object):
    def __init__(self, session):
        self.session = session
        self.model = None
        self.data = None
        self.labels = None
    
    def evaluate_model(self, model, data, labels, attack_names, num_points=10):
        self.model = model
        self.data = data
        self.labels = labels
        
        attack_strengths = np.linspace(0, 0.5, num_points)
        
        losses = np.zeros((len(attack_names), num_points))
        accuracies = np.zeros((len(attack_names), num_points))
        
        for index_name, attack_name in enumerate(attack_names):
            print('Running attack: {}'.format(attack_name))
            for index_strength, attack_strength in enumerate(attack_strengths):
                print('Using attack strength: {}'.format(attack_strength))
                loss, accuracy = self.run_attack(attack_name, attack_strength)
                losses[index_name, index_strength] = loss
                accuracies[index_name, index_strength] = accuracy
            
        return losses, accuracies
    
    def run_attack(self, attack_name, attack_strength):
        wrap = KerasModelWrapper(self.model)
        
        if attack_name is 'basic_iterative':
            attack = BasicIterativeMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default O.3
                             'eps_iter': 0.05,
                             'nb_iter': 10,
                             'y': self.labels,
                             'ord': np.inf,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'carlini_wagner':
            attack = CarliniWagnerL2(wrap, sess=self.session)
            attack_params = {'y': self.labels,
                             'nb_classes': None,
                             'batch_size': 1,
                             'confidence': attack_strength, # Default 0
                             'learning_rate': 0.005,
                             'binary_search_steps': 5,
                             'max_iterations': 1000,
                             'abort_early': True,
                             'initial_const': 0.01,
                             'clip_min': 0,
                             'clip_max': 1}
        elif attack_name is 'deep_fool':
            attack = DeepFool(wrap, sess=self.session)
            attack_params = {'nb_candidate': attack_strength, # Default 10, INT
                             'overshoot': 0.02,
                             'max_iter': 50,
                             'nb_classes': None,
                             'clip_min': 0.0,
                             'clip_max': 1.0}
        elif attack_name is 'elastic_net':
            attack = ElasticNetMethod(wrap, sess=self.session)
            attack_params = {'y': self.labels,
                             'nb_classes': None,
                             'fista': True,
                             'beta': 0.001,
                             'decision_rule': 'EN',
                             'batch_size': 1,
                             'confidence': attack_strength, # Default 0
                             'learning_rate': 0.01,
                             'binary_search_steps': 9,
                             'max_iterations': 1000,
                             'abort_early': False,
                             'initial_const': 0.001,
                             'clip_min': 0,
                             'clip_max': 1}
        elif attack_name is 'fast_feature':
            attack = FastFeatureAdversaries(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'eps_iter': 0.05,
                             'nb_iter': 10,
                             'ord': np.inf,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'fast_gradient':
            attack = FastGradientMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'ord': np.inf,
                             'y': self.labels,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'lbfgs':
            attack = LBFGS(wrap, sess=self.session)
            attack_params = {'batch_size': 1,
                             'binary_search_steps': 5,
                             'max_iterations': 1000,
                             'initial_const': attack_strength, # Default 0.01
                             'clip_min': 0,
                             'clip_max': 1}
        elif attack_name is 'madry':
            attack = MadryEtAl(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'eps_iter': 0.01,
                             'nb_iter': 40,
                             'y': self.labels,
                             'ord': np.inf,
                             'clip_min': None,
                             'clip_max': None,
                             'rand_init': True}
        elif attack_name is 'momentum_iterative':
            attack = MomentumIterativeMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'eps_iter': 0.06,
                             'nb_iter': 10,
                             'y': self.labels,
                             'ord': np.inf,
                             'decay_factor': 1.0,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'spsa':
            attack = SPSA(wrap, sess=self.session)
            attack_params = {'y': self.labels,
                             'epsilon': attack_strength, # Default None
                             'num_steps': None,
                             'is_targeted': False,
                             'early_stop_loss_threshold': None,
                             'learning_rate': 0.01,
                             'delta': 0.01,
                             'batch_size': 128,
                             'spsa_iters': 1,
                             'is_debug': False}
        elif attack_name is 'saliency_map':
            attack = SaliencyMapMethod(wrap, sess=self.session)
            attack_params = {'theta': attack_strength, # Default 1.0
                             'gamma': 1.0,
                             'nb_classes': None,
                             'clip_min': 0.0,
                             'clip_max': 1.0,
                             'symbolic_impl': True}
        elif attack_name is 'virtual_adversarial':
            attack = VirtualAdversarialMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 2.0
                             'num_iterations': 1,
                             'xi': 1e-06,
                             'clip_min': None,
                             'clip_max': None}
        else:
            raise ValueError('Invalid Attack Name!')
        
        adv_x = attack.generate(x, **attack_params)
        data_adv = adv_x.eval(feed_dict={x:self.data}, session=self.session)
        
        score = model.evaluate(data_adv, self.labels, verbose=0)
        loss = score[0]
        accuracy = score[1]
        
        return loss, accuracy

## Impact of Epoch on Security

In [9]:
# Initialize the Fast Gradient Sign Method (FGSM) attack object and graph

gen_adv = GenAdv(sess)

losses = np.zeros((epochs, len(attack_names), num_points))
accuracies = np.zeros((epochs, len(attack_names), num_points))

if not from_saved_model:
    os.mkdir('model_r{}'.format(run_ident))

directory = 'results_r{}'.format(run_ident)
if run_attack and not os.path.exists(directory):
    os.mkdir(directory)
    
for epoch in range(35, 36):
    print('Epoch: {}'.format(epoch+1))
    if from_saved_model:
        model = load_model('model_r{}/model_e{}.h5'.format(run_ident, epoch))
    else:
        model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  initial_epoch=epoch-1,
                  epochs=epoch,
                  verbose=1,
                  validation_data=(X_test, Y_test),
                  callbacks=callbacks)
        model.save('model_r{}/model_e{}.h5'.format(run_ident, epoch))

    if run_attack:
        loss, accuracy = gen_adv.evaluate_model(model, X_test[attack_start:attack_end, :], 
                                                Y_test[attack_start:attack_end], 
                                                attack_names, num_points=num_points)
        losses[epoch,:,:] = loss
        accuracies[epoch,:,:] = accuracy
        
        np.save('results_r{}/loss_e{}.npy'.format(run_ident, epoch), losses)
        np.save('results_r{}/accuracy_e{}.npy'.format(run_ident, epoch), accuracies)

Epoch: 32
Running attack: basic_iterative
Using attack strength: 0.0
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Using attack strength: 0.05555555555555555
Using attack strength: 0.1111111111111111
Using attack strength: 0.16666666666666666
Using attack strength: 0.2222222222222222
Using attack strength: 0.2777777777777778
Using attack strength: 0.3333333333333333
Using attack strength: 0.38888888888888884
Using attack strength: 0.4444444444444444
Using attack strength: 0.5
Running attack: fast_gradient
Using attack strength: 0.0
Using attack strength: 0.05555555555555555
Using attack strength: 0.1111111111111111
Using attack strength: 0.16666666666666666
Using attack strength: 0.2222222222222222
Usi

In [10]:
if run_attack:
    np.save('loss_r{}.npy'.format(run_ident), losses)
    np.save('accuracy_r{}.npy'.format(run_ident), accuracies)
    max_loss = losses.flatten().max()
    max_accuracy = accuracies.flatten().max()

In [11]:
def plot_func(epoch):
    fig = plt.figure(figsize=(8, 6))
    plt.subplot(121)
    for index, attack_name in enumerate(attack_names):
        x_plt = np.linspace(0, 0.5, num_points)
        y_plt = losses[epoch, index, :].flatten()
        plt.plot(x_plt, y_plt, label=attack_name)
    plt.title('Adversarial Loss')
    plt.xlabel('Attack Strength')
    plt.ylabel('Loss')
    plt.ylim(ymax=max_loss)
    plt.legend()

    plt.subplot(122)
    for index, attack_name in enumerate(attack_names):
        x_plt = np.linspace(0, 0.5, num_points)
        y_plt = accuracies[epoch, index, :].flatten()
        plt.plot(x_plt, y_plt, label=attack_name)
    plt.title('Adversarial Accuracy')
    plt.xlabel('Attack Strength')
    plt.ylabel('Accuracy')
    plt.ylim(ymax=max_accuracy)
    plt.legend()
    plt.show()

In [12]:
if run_attack:
    interact(plot_func, epoch=widgets.IntSlider(min=0,max=epochs-1,step=1,value=0));

interactive(children=(IntSlider(value=0, description='epoch', max=99), Output()), _dom_classes=('widget-interact',))